In [ ]:
# Load EDA Pkgs
import pandas as pd
import neattext.functions as nt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [ ]:
data= pd.read_csv('/content/drive/MyDrive/Course_Recommender/udemy_courses.csv')

In [ ]:
data.tail()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,clean_course_title
3673,775618,Learn jQuery from Scratch - Master of JavaScri...,https://www.udemy.com/easy-jquery-for-beginner...,True,100,1040,14,21,All Levels,2.0,2016-06-14T17:36:46Z,Web Development,Learn jQuery Scratch Master JavaScript library
3674,1088178,How To Design A WordPress Website With No Codi...,https://www.udemy.com/how-to-make-a-wordpress-...,True,25,306,3,42,Beginner Level,3.5,2017-03-10T22:24:30Z,Web Development,Design WordPress Website Coding
3675,635248,Learn and Build using Polymer,https://www.udemy.com/learn-and-build-using-po...,True,40,513,169,48,All Levels,3.5,2015-12-30T16:41:42Z,Web Development,Learn Build Polymer
3676,905096,CSS Animations: Create Amazing Effects on Your...,https://www.udemy.com/css-animations-create-am...,True,50,300,31,38,All Levels,3.0,2016-08-11T19:06:15Z,Web Development,CSS Animations Create Amazing Effects Website
3677,297602,Using MODX CMS to Build Websites: A Beginner's...,https://www.udemy.com/using-modx-cms-to-build-...,True,45,901,36,20,Beginner Level,2.0,2014-09-28T19:51:11Z,Web Development,MODX CMS Build Websites Beginners Guide


In [ ]:
data['clean_course_title'] = data['course_title'].apply(nt.remove_stopwords)

In [ ]:
# Clean Text:stopwords,special charac
data['clean_course_title'] = data['clean_course_title'].apply(nt.remove_special_characters)

In [ ]:
data[['course_title','clean_course_title']]

,course_title,clean_course_title
0,Ultimate Investment Banking Course,Ultimate Investment Banking Course
1,Complete GST Course & Certification - Grow You...,Complete GST Course Certification Grow Practice
2,Financial Modeling for Business Analysts and C...,Financial Modeling Business Analysts Consultants
3,Beginner to Pro - Financial Analysis in Excel ...,Beginner Pro Financial Analysis Excel 2017
4,How To Maximize Your Profits Trading Options,Maximize Profits Trading Options
...,...,...
3673,Learn jQuery from Scratch - Master of JavaScri...,Learn jQuery Scratch Master JavaScript library
3674,How To Design A WordPress Website With No Codi...,Design WordPress Website Coding
3675,Learn and Build using Polymer,Learn Build Polymer
3676,CSS Animations: Create Amazing Effects on Your...,CSS Animations Create Amazing Effects Website


In [ ]:
# Vectorize our Text
count_vect = CountVectorizer()
cv_mat = count_vect.fit_transform(data['clean_course_title'])

In [ ]:
df_cv_words = pd.DataFrame(cv_mat.todense(),columns=count_vect.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df_cv_words.head()

,000005,001,01,02,10,100,101,101master,102,10k,...,zend,zero,zerotohero,zf2,zinsen,zoho,zombie,zu,zuhause,zur
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Cosine Similarity Matrix
cosine_sim_mat = cosine_similarity(cv_mat)

In [ ]:
cosine_sim_mat

array([[1.        , 0.20412415, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20412415, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.23570226],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.23570226, 0.        ,
        1.        ]])

In [ ]:
# function for content based recommender from the above cosine similiarity
def content_based_recommender(title, cosine_sim, dataframe):
    
    # Create indexes
    indices = pd.Series(data.index, index=data['course_title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    
    # Catch the title of index
    course_index = indices[title]
    
    # Calculate the similarity score based on title
    similarity_scores = pd.DataFrame(cosine_sim_mat[course_index], columns=['score'])
    
    # Recommend 5 courses
    movie_indices = similarity_scores.sort_values('score', ascending=False)
    movie_indices = movie_indices[movie_indices['score'] > 0.25][1:10].index
    
    return data['course_title'].iloc[movie_indices]

In [ ]:
#test 1
content_based_recommender('Complete Python Web Course: Build 8 Python Web Apps',cosine_sim_mat,data)

3418    Build Web Apps Using EmberJS: The Complete Course
2497                          Web Programming with Python
3343                               Python Web Programming
2673    The Bubble Course - Create Complete Web Apps w...
3141    The Complete Web Development Course -  Build 1...
3658    The Complete Web Developer Course - Build 20 W...
2736          The Original Complete Web Developer Course.
3232                The Complete Web Developer Course 2.0
2974         Build Apps with ReactJS: The Complete Course
Name: course_title, dtype: object

In [ ]:
#test2
content_based_recommender('Ultimate Investment Banking Course',cosine_sim_mat,data)

39            The Complete Investment Banking Course 2017
3474                           The Ultimate jQuery Course
2714                  The Ultimate Web Development Course
240            Advanced Accounting for Investment Banking
2802                     Ultimate WordPress Plugin Course
417             The Investment Banking Recruitment Series
1211    The Ultimate Drawing Course - Beginner to Adva...
2643              The Ultimate Vue JS 2 Developers Course
3003    Ultimate CSS3 Course - From Beginners To Profe...
Name: course_title, dtype: object